
#Ingest data

In [0]:
%sql

USE CATALOG supply_chain;
CREATE SCHEMA IF NOT EXISTS analytics;
USE SCHEMA analytics;

In [0]:
shipment_df = spark.read.option("header", "true").option("inferschema", "true").csv("/Volumes/supply_chain/analytics/raw/supply_chain_data.csv")
shipment_df.createOrReplaceTempView("uvw_src_shipments")

In [0]:
%sql

CREATE OR REPLACE TABLE shipments (
  product_type STRING
  , sku STRING
  , price FLOAT
  , availability INT
  , number_of_products_sold INT
  , revenue_generated FLOAT
  , customer_demographics STRING
  , stock_levels INT
  , lead_times INT
  , order_quantities INT
  , shipping_times INT
  , shipping_carriers STRING
  , shipping_costs FLOAT
  , supplier_name STRING
  , location STRING
  , lead_time INT
  , production_volumes INT
  , manufacturing_lead_time INT
  , manufacturing_costs FLOAT
  , inspection_results STRING
  , defect_rates FLOAT
  , transportation_modes STRING
  , routes STRING
  , costs FLOAT
)

In [0]:
%sql

INSERT INTO shipments
SELECT 
  *
FROM uvw_src_shipments


#1. Total quantity shipped for each product category

In [0]:
%sql

SELECT
  product_type
  , SUM(order_quantities)      AS total_quantity_shipped
FROM shipments
GROUP BY product_type


#2. Warehouses with most efficient shipping processes based on average shipping times

In [0]:
%sql

SELECT
  location    AS Warehouse
  , ROUND(AVG(shipping_times), 2)               AS average_shipping_times
  , RANK() OVER (ORDER BY AVG(shipping_times) DESC) AS rank
FROM shipments
GROUP BY location


#3. Total value of shipments for each supplier

In [0]:
%sql

SELECT
  supplier_name
  , ROUND(SUM(price*order_quantities), 2)     AS total_value_of_shipments
FROM shipments
GROUP BY supplier_name
ORDER BY supplier_name


#4. Top 5 products with highest total shipment quantities

In [0]:
%sql

SELECT
  sku                                                     AS product
  , SUM(order_quantities)                                 AS total_quantity_shipped
  , RANK() OVER(ORDER BY SUM(order_quantities) DESC)      AS rank
FROM shipments
GROUP BY sku
LIMIT 5


# Distribution of shipment values for each product category

In [0]:
%sql

SELECT
  product_type
  , ROUND(SUM(price*order_quantities), 2)     AS total_value_of_shipments
FROM shipments
GROUP BY product_type
ORDER BY total_value_of_shipments DESC